# Segmenting and Clustering Neighborhoods in New York City


### Third
1. Explore and cluster the neighborhoods in Toronto 
2. Submit a link to the new Notebook on your Github repository

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


In [3]:
# Use the Notebook to build the code to scrape the Wikipedia page
table_TO = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match='Postal Code')
df_TO = table_TO[0]
df_TO.shape


(180, 3)

In [4]:
# Delete Borough equal Not assigned
indexBorough = df_TO[df_TO['Borough'] == 'Not assigned' ].index
df_TO.drop(indexBorough , inplace=True)
df_CA = df_TO.reset_index(drop=True)

In [5]:
# Change value Neighbourhood to Borough if Neighbourhood is equal Not assigned
indexNeighbour = df_CA[df_CA['Neighbourhood'] == 'Not assigned' ].index
for i in range(len(indexNeighbour)): df_CA['Neighbourhood'].loc[i] = df_CA['Borough'].loc[i]
df_CA.shape

(103, 3)

In [13]:
#Separe Neighbourhoods
try:
    del df_CA2
except:
    pass
df_CA2 = pd.DataFrame(columns=['Postal Code','Borough','Neighbourhood'])
for x in range(len(df_CA.index)): 
    try:
        if df_CA['Neighbourhood'].loc[x].find(',') >= 1 :
            neigh = df_CA['Neighbourhood'].loc[x].split(',')
            y = 0
            for y in range(len(neigh[y:])): 
                df_CA2 = df_CA2.append({"Postal Code": df_CA['Postal Code'].loc[x], "Borough": df_CA['Borough'].loc[x], \
                                        "Neighbourhood": neigh[y]}, ignore_index=True)
            df_CA = df_CA.drop([x])
        else:
            pass
    except:
        pass
else:
    pass

# Add Neighbourhood in df_CA
df_CA = df_CA.append(df_CA2)
df_CA = df_CA.drop_duplicates() 
df_CA = df_CA.reset_index(drop=True)
print('Table size.......: ', df_CA.shape)

Table size.......:  (217, 3)


In [14]:
# Importr CSV with the geographical coordinates
df_GEO = pd.read_csv('https://cocl.us/Geospatial_data')

In [16]:
# Creat New dataframe with geographical coordinates
try:
    del df_CAGEO
except:
    pass
df_CAGEO = pd.DataFrame(columns=['Postal Code','Borough','Neighbourhood','Latitude','Longitude'])
for i in range(len(df_CA.index)):
    check = True
    j = 0
    while (j < len(df_GEO.index)) and (check):
        if df_CA['Postal Code'].loc[i] == df_GEO['Postal Code'].loc[j]:
            df_CAGEO = df_CAGEO.append({"Postal Code": df_CA['Postal Code'].loc[i], "Borough": df_CA['Borough'].loc[i], \
                                      "Neighbourhood": df_CA['Neighbourhood'].loc[i],"Latitude": df_GEO['Latitude'].loc[j],\
                                      "Longitude": df_GEO['Longitude'].loc[j]}, ignore_index=True)
            check = False
        else:
            j = j + 1
else:
    pass     
    

(217, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M3B,North York,Don Mills,43.745906,-79.352188
3,M6B,North York,Glencairn,43.709577,-79.445073
4,M3C,North York,Don Mills,43.725900,-79.340923
